# Import Library and project credentials

In [4]:
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import pdfplumber
import numpy as np
PROJECT_ID = "walmart-retail-media"
MODEL_ID = "text-embedding-004"

# Pre-trained Embedding model text-embedding-004 to create embedding 

In [5]:

class ExtractCreateEmbeddings:

    def __init__(self, file_path):
        self.model = TextEmbeddingModel.from_pretrained("text-embedding-004")
        self.path = file_path
        print("PAth:::   ", self.path)


    def embed_text(self, text: list):
        embeddings = self.model.get_embeddings([text])
        res = []
        for embedding in embeddings:
            res.append(embedding.values)
            # print(len(embedding.values))
        # print(len(res))
        return res
   

    # Function to extract text from PDF
    def extract_text_from_pdf(self):
        # Open the PDF file
        with pdfplumber.open(self.path) as pdf:
            all_text = ""
            extract = []
            # Iterate over all the pages
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()  # Extract text from each page
                if text:
                    embed = self.embed_text(text)
                    # extract.append([text,embed[0]])
                    # extract[f"page_{i+1}"] = [text, embed[0]]
                    extract.append([text, embed[0]])
                print(f"Extracted text from page {i + 1}")

            return extract
        
def embed_text(text: list):
        model = TextEmbeddingModel.from_pretrained("text-embedding-004")
        embeddings = model.get_embeddings([text])
        res = []
        for embedding in embeddings:
            res.append(embedding.values)
            # print(len(embedding.values))
        # print(len(res))
        return res


# creating embedding for the uploaded pdf file

In [9]:
embed_values[1][0]

'Contents\nSr No Content Page number\nWelcome and general guidelines\nBusiness Ethics\n1 Code of Conduct 4-10\nBenefits\n2 Leave Policy 11-20\n3 Expense Reimbursement 21-23\n4 Relocation Policy – Within India 24 – 25\n5 Relocation Policy – Any country to India 26 – 27\nTredence Academy of Lots of Learning\n6 U Learn V Pay 28 – 30\nReward & Performance\n7 Reward and Recognition 31-35\n8 Performance Management 36-37\nCompliance\n9 Privacy and Confidentiality Policy 38\n10 Intellectual property Policy 39-42\n11 Conflict of Interest Policy 43-45\n12 POSH Policy 46-55\nOthers\n13 Exit Policy 56-58\nAnnexure\n14 Acceptable Usage Policy (AUP) 60-78\n1\nEmployee Handbook 2024-India | TRE-ISMS-DP-001 | CONFIDENTIAL'

In [10]:
file_path=r"C:\Users\manjuprasad.n\Downloads\India Employee Handbook - 2024.pdf"
embed = ExtractCreateEmbeddings(file_path)
embed_values= embed.extract_text_from_pdf()  
embedded_vectors,text = [],[]
for i in range(len(embed_values)):
    embedded_vectors.append(embed_values[i][1])
    text.append(embed_values[i][0])

np_vectors = np.array(embedded_vectors)





PAth:::    C:\Users\manjuprasad.n\Downloads\India Employee Handbook - 2024.pdf
Extracted text from page 1
Extracted text from page 2
Extracted text from page 3
Extracted text from page 4
Extracted text from page 5
Extracted text from page 6
Extracted text from page 7
Extracted text from page 8
Extracted text from page 9
Extracted text from page 10
Extracted text from page 11
Extracted text from page 12
Extracted text from page 13
Extracted text from page 14
Extracted text from page 15
Extracted text from page 16
Extracted text from page 17
Extracted text from page 18
Extracted text from page 19
Extracted text from page 20
Extracted text from page 21
Extracted text from page 22
Extracted text from page 23
Extracted text from page 24
Extracted text from page 25
Extracted text from page 26
Extracted text from page 27
Extracted text from page 28
Extracted text from page 29
Extracted text from page 30
Extracted text from page 31
Extracted text from page 32
Extracted text from page 33
Extrac

# Creating collection in Qdrant DB

In [12]:
from qdrant_client import QdrantClient, models

# Connect to Qdrant (local instance)
client = QdrantClient(url="http://localhost:6333")

# Define collection name
collection_name = "my_cosine_collection"

In [16]:
# Create a new collection with 100-dimensional vectors and Cosine similarity
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

print(f"Collection '{collection_name}' created successfully!")


Collection 'my_cosine_collection' created successfully!


# Storing vector embedding in qdrant db collection

In [17]:
from qdrant_client import QdrantClient

# Initialize Qdrant client
client = QdrantClient(url="http://localhost:6333")

collection_name = "my_cosine_collection"

In [ ]:
# Insert embeddings into Qdrant
client.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(id=i, vector=embedded_vectors[i], payload={"text": text[i]})
        for i in range(len(embedded_vectors))
    ]
) 

# Creating embedding for questions 

In [22]:
questions=['Types of expenses covered for relocation pplicy within India',
           'What are half yearly awards',
           'What is the criteria for PAT on the back']

In [23]:
q_embed=[]
from rag.create_embedding import embed_text
for q in questions:
    target_vector = np.array(embed_text(q))
    q_embed.append(target_vector)

# Finding similary b/w responses and questions

In [24]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from utils.config import cos_top_n, knn_top_n


class SimilaritySearch:

    def __init__(self):
        self.top_n = cos_top_n
        # Initialize the NearestNeighbors model with cosine similarity
        self.knn_model = NearestNeighbors(n_neighbors=knn_top_n, metric='manhattan')

    # Function to perform KNN similarity search using cosine similarity
    def knn_similarity_search(self, target_vector, vectors):
        # Fit the model on the dataset of vectors
        self.knn_model.fit(vectors)
        
        # Reshape target vector to 2D (as required by scikit-learn)
        target_vector = np.array(target_vector).reshape(1, -1)
        
        # Perform the KNN search
        distances, indices = self.knn_model.kneighbors(target_vector)
        
        # Return the indices of the nearest vectors and their distances
        return indices.flatten(), distances.flatten()


    # Function to compute cosine similarity and return top N most similar vectors
    def cosine_similarity_search(self, target_vector, vectors):
        # Reshape target_vector to 2D since cosine_similarity expects 2D arrays
        target_vector = np.array(target_vector).reshape(1, -1)
        
        # Compute cosine similarities between the target vector and all other vectors
        similarities = cosine_similarity(vectors, target_vector)
        
        # Flatten the result array
        similarities = similarities.flatten()

        # Get indices of top N most similar vectors (in descending order of similarity)
        top_n_indices = similarities.argsort()[::-1][:self.top_n]
        
        # Get corresponding similarity scores
        top_n_similarities = similarities[top_n_indices]
        return top_n_indices, top_n_similarities
    


    def search(self, target_vector, vectors):
        
        knn_top_indices, knn_top_distances = self.knn_similarity_search(target_vector, vectors)
        cos_top_indices, cos_top_similarities = self.cosine_similarity_search(target_vector, vectors)

       
        
        return cos_top_indices,cos_top_similarities,knn_top_indices,knn_top_distances


In [26]:
formatted_output = []
ss=SimilaritySearch()
context_pages_ouput=[]
for qr in range(len(questions)):  
    cos_top_indices, cos_top_similarities, knn_top_indices, knn_top_distances = ss.search(target_vector=q_embed[qr], vectors=np_vectors)
    output = [questions[qr]] 
    context_pages = set()
    for i in cos_top_indices:
        context_pages.add(i)
    for i in knn_top_indices:
            context_pages.add(i)
    for idx, similarity in zip(cos_top_indices, cos_top_similarities):
        output.append(f"{idx},{similarity}")
    for idx, distance in zip(knn_top_indices, knn_top_distances):
        output.append(f"{idx},{distance}")
    formatted_output.append(output)
    context_pages_ouput.append(context_pages)
print(formatted_output)


[['Types of expenses covered for relocation pplicy within India', '24,0.73167514891693', '26,0.7234795860880974', '25,0.7206877359265444', '26,16.31082082206285', '24,16.39321315582947', '25,16.624978481821927'], ['What are half yearly awards', '33,0.6012876925602934', '34,0.5745539086034994', '32,0.5488804145321222', '33,19.63542295501611', '34,20.477088664374605', '32,21.018170781212348'], ['What is the criteria for PAT on the back', '36,0.4221254111180226', '58,0.38948894705687287', '33,0.38037380928989345', '36,23.795754771310385', '58,24.214489592352038', '33,24.60175469755268']]


In [7]:
import pandas as pd
# df_=pd.DataFrame(formatted_output)
columns = ['question', 'cos1', 'cos2', 'cos3', 'knn1', 'knn2', 'knn3']

# Create the DataFrame
df = pd.DataFrame(formatted_output, columns=columns)

# Display the DataFrame
display(df)

,question,cos1,cos2,cos3,knn1,knn2,knn3
0,Types of expenses covered for relocation pplic...,"24,0.73167514891693","26,0.7234795860880974","25,0.7206877359265444","26,16.31082082206285","24,16.39321315582947","25,16.624978481821927"
1,What are half yearly awards,"33,0.6012876925602934","34,0.5745539086034994","32,0.5488804145321222","33,19.63542295501611","34,20.477088664374605","32,21.018170781212348"
2,What is the criteria for PAT on the back,"36,0.4221254111180226","58,0.38948894705687287","33,0.38037380928989345","36,23.795754771310385","58,24.214489592352038","33,24.60175469755268"
